In [ ]:
!pip install torchsummary

In [ ]:
import os
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from torchvision.utils import save_image, make_grid
import matplotlib.pyplot as plt
from torchsummary import summary

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(1)


In [ ]:
# Parameters
batch_size = 64
latent_dim = 100  # Dimension of the noise vector
image_shape = (3, 64, 64)  # Shape of images (3 channels, 64x64 resolution)
sample_fraction = 0.4  # Using 40% of the dataset


In [ ]:
# Define transformations
train_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.5] * 3, [0.5] * 3)  # Normalization for GAN training
])

# Custom dataset class
class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.image_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(('jpg', 'jpeg', 'png'))]

        # Limit the dataset to 40% of the images
        subset_size = int(len(self.image_files) * sample_fraction)
        self.image_files = np.random.choice(self.image_files, subset_size, replace=False)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_path = self.image_files[idx]
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image

# Initialize dataset and DataLoader
train_data_path = '/kaggle/input/selfie2anime/trainB'
train_dataset = ImageDataset(folder_path=train_data_path, transform=train_transform)
dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


In [ ]:
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.ConvTranspose2d(latent_dim, 512, kernel_size=4, stride=1, padding=0),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, z):
        return self.model(z)

# Instantiate and move to device
generator = Generator(latent_dim).to(device)
summary(generator, (latent_dim, 1, 1))


In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=0),
            nn.Sigmoid()
        )

    def forward(self, img):
        return self.model(img).view(-1, 1).squeeze(1)

# Instantiate and move to device
discriminator = Discriminator().to(device)
summary(discriminator, (3, 64, 64))


In [ ]:
# Loss function
adversarial_loss = nn.BCELoss()

# Optimizers
lr = 0.0002
betas = (0.5, 0.999)
optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=betas)
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=betas)


In [ ]:
def generate_and_save_images(epoch, fixed_noise):
    generator.eval()
    with torch.no_grad():
        generated_images = generator(fixed_noise).detach().cpu()
    grid = make_grid(generated_images, nrow=8, normalize=True)
    save_image(grid, f"generated_images_epoch_{epoch}.png")
    plt.imshow(grid.permute(1, 2, 0))
    plt.axis("off")
    plt.show()


In [ ]:
d_losses = []
g_losses = []
from PIL import Image

num_epochs = 250
fixed_noise = torch.randn(64, latent_dim, 1, 1, device=device)  # Fixed noise for consistent image generation

for epoch in range(num_epochs):
    for i, imgs in enumerate(dataloader):  # Modified to only unpack imgs
        # Prepare real and fake data
        real_imgs = imgs.to(device)
        real_labels = torch.ones(imgs.size(0), device=device)  # Changed to match discriminator output size
        fake_labels = torch.zeros(imgs.size(0), device=device)  # Changed to match discriminator output size
        
        # ---------------------
        #  Train Discriminator
        # ---------------------
        optimizer_D.zero_grad()
        
        # Real images
        real_loss = adversarial_loss(discriminator(real_imgs), real_labels)
        
        # Fake images
        z = torch.randn(imgs.size(0), latent_dim, 1, 1, device=device)
        fake_imgs = generator(z)
        fake_loss = adversarial_loss(discriminator(fake_imgs.detach()), fake_labels)
        
        # Discriminator loss
        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        optimizer_D.step()
        
        # -----------------
        #  Train Generator
        # -----------------
        optimizer_G.zero_grad()
        
        # Generate fake images
        fake_imgs = generator(z)
        g_loss = adversarial_loss(discriminator(fake_imgs), real_labels)
        
        # Generator loss
        g_loss.backward()
        optimizer_G.step()
        
        # Log progress
        if i % 100 == 0:
            print(f"Epoch [{epoch}/{num_epochs}] Batch [{i}/{len(dataloader)}] "
                  f"D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}")
    
    # Save generated images every epoch
    generate_and_save_images(epoch, fixed_noise)


In [ ]:
import torch

# Assuming generator and discriminator are your model instances
torch.save(generator.state_dict(), "generator.pth")
torch.save(discriminator.state_dict(), "discriminator.pth")
